`Rubicon_ml` makes it easy to log plots with features and artifacts. In this example we'll walk through creating a feature dependency plot using the `shap` package and saving it to an artifact.

First lets create a Rubicon `project` and fit a `pipeline`.

In [ ]:
import shap
import sklearn
from sklearn.datasets import load_wine
from rubicon_ml import Rubicon

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from rubicon_ml.sklearn import make_pipeline

rubicon = Rubicon(persistence="memory")
project = rubicon.get_or_create_project("Logging Feature Plots")

X, y = load_wine(return_X_y=True)

reg =  GradientBoostingRegressor(random_state=1)
pipeline=make_pipeline(project, reg)

pipeline.fit(X, y)

## Generating Data
Using `shap.Explainer` we can generate `shap` values to later plot.

In [ ]:
explainer = shap.Explainer(reg)
shap_values = explainer.shap_values(X)

## Plotting

In [ ]:
import matplotlib.pyplot as pl
import io

In [ ]:
buff_data = dict()
for x in range(reg.n_features_in_):
    feature_name="feature "+str(x)
    buf = io.BytesIO()
    exp=project.log_experiment()
    exp.log_feature(feature_name)
    shap.dependence_plot(x, shap_values, X, interaction_index=None)
    fig = pl.gcf()
    fig.savefig(buf, format="png")
    buf.seek(0)
    buff_data[feature_name] = buf.read()
    buf.close()
    exp.log_artifact(data_bytes=buff_data[feature_name],name=feature_name+" dependency plot")
    exp.add_tags([feature_name])
    



In [ ]:
import io
from PIL import Image

exp=project.experiments(tags=["feature 6"])[0]
artifact_plot=exp.artifact("feature 6 dependency plot")
imageScatterPlotStream = io.BytesIO(artifact_plot.data)
scatter_plot_image = Image.open(imageScatterPlotStream)
display(scatter_plot_image)